In [2]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
import shutil
import glob


# Get data from icdar 
80% training and 20% test

In [5]:
def dividing_data():
    
  files = os.listdir("our dataset/data/")
  X=[]
  for file in files:
    X.append("our dataset/data/"+file)

  train, test = train_test_split(X, test_size=0.2) 

  # delete all imgs in folder to replace it with a new data
  if not os.path.exists("our dataset/train/"):
       os.makedirs("our dataset/train")
  
  if not os.path.exists("our dataset/test/"):
       os.makedirs("our dataset/test")

  for i in os.listdir("our dataset/train/"):
    os.remove("our dataset/train/"+i)
  
  for i in os.listdir("our dataset/test/"):
    os.remove("our dataset/test/"+i)

    
  # save imgs to train and test folders 
  dst_dir = "our dataset/train/"
  for src_dir in train:
    for img in glob.iglob(src_dir):
        shutil.copy(img, dst_dir)
  
  dst_dir = "our dataset/test/"
  for src_dir in test:
    for img in glob.iglob(src_dir):
        shutil.copy(img, dst_dir)
  

In [16]:
def cropTxtOnly(image):
    img = image[30:-450,20:-20] # Perform pre-cropping

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # gray = 255*(gray < 90).astype(np.uint8) # To invert the text to white

    gray = 255*(gray < 200).astype(np.uint8) # To invert the text to white

    gray = cv2.morphologyEx(gray, cv2.MORPH_OPEN, np.ones((2, 2), dtype=np.uint8)) # Perform noise filtering

    coords = cv2.findNonZero(gray) # Find all non-zero points (text)
    x, y, w, h = cv2.boundingRect(coords) # Find minimum spanning bounding box
    rect = img[y:y+h, x:x+w] # Crop the image - note we do this on the original image

    return rect


In [23]:

def preprocessing(dataSrc, target):
    
    files=os.listdir("our dataset/"+dataSrc)

    if not os.path.exists(target):
          os.makedirs(target)
      
    for i in os.listdir(target):
          os.remove(target+i)
          
    for file in files:
        foldername=file.split('.')[0]
        
        # load image
        img = cv2.imread("our dataset/"+dataSrc+ file )

        #crop the text only from the image
        cropped_img=cropTxtOnly(img)
        
        # convert to gray
        gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
            
        # threshold the grayscale image
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        # use morphology erode to blur horizontally
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (200, 3))
        morph = cv2.morphologyEx(thresh, cv2.MORPH_DILATE, kernel)
        # use morphology open to remove thin lines from dotted lines
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 17))
        morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)
        # find contours
        cntrs = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cntrs = cntrs[0] if len(cntrs) == 2 else cntrs[1]
        # write result to disk
        cv2.imwrite(target+foldername+".jpg", thresh)

# Preprocessing data

In [7]:
dividing_data() #80% train and 20% test

In [24]:
preprocessing(dataSrc="train/",target="Training_data/")


In [ ]:
preprocessing(dataSrc="test/",target="Test_data/")